<a href="https://colab.research.google.com/github/gyang22/algorithmic_trading_project/blob/main/simple_stock_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0 | Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

## 1 | Define the model

In [ ]:
class StockClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(StockClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

## 2 | Preprocess data & performing train test split

2.1 | Loading and viewing data

In [ ]:
data = pd.read_csv('stock_data.csv') # replace with csv
data.head()

NameError: ignored

2.2 | Cost Column

In [ ]:
# assuming there are these columns
data['Cost'] = data['Gains'] - data['Losses']
data['Label'] = np.where(data['Cost'] > 0, 1, 0) # if cost > 0, buy (1), otherwise sell (0)

2.3 | Train Test Split

In [ ]:
X = data.drop(['Label'], axis=1).values
Y = data['Label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## 3 | Loading pytorch

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float)
Y_train = torch.tensor(Y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float)
Y_test = torch.tensor(Y_test, dtype=torch.long)

# PyTorch DataLoader
train_data = TensorDataset(X_train, Y_train)
test_data = TensorDataset(X_test, Y_test)
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

# Instantiate the model
model = StockClassifier(input_size=X.shape[1], hidden_size=64, num_classes=2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## 4 | Training

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    for i, (stocks, labels) in enumerate(train_loader):
        outputs = model(stocks)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

## 5 | Testing

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for stocks, labels in test_loader:
        outputs = model(stocks)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the test stocks: {100 * correct / total}%')